In [2]:

class TentaclePlanner:
    def __init__(
        self,
        obstacles  = [],
        dt = 0.1,
        steps = 5, # Number of steps the planner uses when simulating a path
        alpha = 1, # Weighs the positional error in the cost function. Higher value prioritises goal positions.
        beta = 0.1, # Weighs the rotational error of the cost function. higher value prioritises correct final orientation.
        max_linear_velocity = 0.23, # 0.23 m/s is at about 75% duty cycle.
        max_angular_velocity = 2.1, # 2.1 rad/s is about 78% duty cycle
        max_linear_tolerance = 0.1, # meters
        max_angular_tolerance = 0.1, # radians, 0.15 rad = 8.6 deg, 0.08 = 4.6 deg
        max_acceleration = 0.1,
        max_angular_acceleration=0.2,
        current_linear_velocity = 0.0,
        current_angular_velocity = 0.0
    ):
        self.dt = dt
        self.steps = steps
        self.alpha = alpha
        self.beta = beta
        self.obstacles = np.array(obstacles)
        self.max_linear_velocity = max_linear_velocity
        self.max_angular_velocity = max_angular_velocity
        self.max_linear_tolerance = max_linear_tolerance
        self.max_angular_tolerance = max_angular_tolerance 
        self.max_acceleration = max_acceleration
        self.max_angular_acceleration = max_angular_acceleration
        self.current_linear_velocity = current_linear_velocity
        self.current_angular_velocity = current_angular_velocity
        
        self.tentacles = self._generate_tentacles()

    def _generate_tentacles(self):
        linear_velocities = np.linspace(-self.max_linear_velocity, self.max_linear_velocity, 5)
        angular_velocities = np.linspace(-self.max_angular_velocity, self.max_angular_velocity, 5)
        tentacles = [(v, w) for v in linear_velocities for w in angular_velocities]
        pure_linear_tentacles = [(v, 0.0) for v in linear_velocities if v != 0] # pure linear motion options
        tentacles.extend(pure_linear_tentacles)
        pure_angular_tentacles = [(0.0, w) for w in angular_velocities if w != 0]
        tentacles.extend(pure_angular_tentacles)
        tentacles.append((0.0, 0.0))  # Add stop option
        tentacles = list(set(tentacles))

        return tentacles
    
    """I thought the issue was tentacle generation but I don't think this is the case as this just made it worse.."""
    def _generate_enhanced_tentacles(self):
        # Increase the number of linear and angular velocity options
        linear_velocities = np.linspace(-self.max_linear_velocity, self.max_linear_velocity, 11)
        angular_velocities = np.linspace(-self.max_angular_velocity, self.max_angular_velocity, 11)

        # Generate basic tentacles
        tentacles = [(v, w) for v in linear_velocities for w in angular_velocities]

        # Add more granular options around zero velocity
        fine_linear = np.linspace(-0.1 * self.max_linear_velocity, 0.1 * self.max_linear_velocity, 5)
        fine_angular = np.linspace(-0.1 * self.max_angular_velocity, 0.1 * self.max_angular_velocity, 5)
        tentacles.extend([(v, w) for v in fine_linear for w in fine_angular])

        # Add pure linear motions
        pure_linear = [(v, 0) for v in linear_velocities if v != 0]
        tentacles.extend(pure_linear)

        # Add pure rotations
        pure_rotations = [(0, w) for w in angular_velocities if w != 0]
        tentacles.extend(pure_rotations)

        # Add diagonal motions
        diagonals = [(v, v * self.max_angular_velocity / self.max_linear_velocity) for v in linear_velocities if v != 0]
        diagonals.extend([(v, -v * self.max_angular_velocity / self.max_linear_velocity) for v in linear_velocities if v != 0])
        tentacles.extend(diagonals)

        # Add stop option
        tentacles.append((0, 0))

        # Remove duplicates and return
        return list(set(tentacles))
    
    
    def is_goal_reached(self, goal_x, goal_y, goal_th, x, y, th):
        distance_to_goal = np.hypot(goal_x - x, goal_y - y)
        angular_error = np.arctan2(np.sin(goal_th - th), np.cos(goal_th - th))
        return distance_to_goal <= self.max_linear_tolerance and abs(angular_error) <= self.max_angular_tolerance

    def trapezoidal_profile(self, distance, current_velocity, max_velocity, max_acceleration):
        acceleration_distance = (max_velocity ** 2) / (2 * max_acceleration)
        if distance == 0:
            velocity = current_velocity + max_acceleration * self.dt
        elif 0 < distance <= 2 * acceleration_distance:
            t_accel = np.sqrt(distance / max_acceleration)
            velocity = min(current_velocity + max_acceleration * t_accel, max_velocity)
        else:
            velocity = max_velocity
        return velocity

    def _roll_out(self, v, w, goal_x, goal_y, goal_th, x, y, th):
        for _ in range(self.steps):
            x += self.dt * v * np.cos(th)
            y += self.dt * v * np.sin(th)
            th += w * self.dt

            if self._check_collision(x, y):
                return np.inf

        e_th = np.arctan2(np.sin(goal_th - th), np.cos(goal_th - th))
        cost = self.alpha * ((goal_x - x) ** 2 + (goal_y - y) ** 2) + self.beta * (e_th ** 2)
        return cost

    def _check_collision(self, x, y):
        if len(self.obstacles) == 0:
            return False
        min_dist = np.min(np.hypot(x - self.obstacles[:, 0], y - self.obstacles[:, 1]))
        return min_dist < 0.1

    def _angle_between_points(self, p1, p2):
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        return np.arctan2(dy, dx)
    
    
    def _normalise_angle(self, angle):  # in radians
        return (angle + np.pi) % (2 * np.pi) - np.pi
    
    def _shortest_angular_distance(self, from_angle, to_angle):
        """
        Calculate the shortest angular distance between two angles.
        """
        diff = self._normalise_angle(to_angle - from_angle)
        if diff > np.pi:
            diff -= 2 * np.pi
        return diff
        
    def _plan_tentacles(self, goal_x, goal_y, goal_th, x, y, th):
        if self.is_goal_reached(goal_x, goal_y, goal_th, x, y, th):
#             print("Goal is within tolerance, stopping planner function")
            return (0.0, 0.0)

        costs = [self._roll_out(v, w, goal_x, goal_y, goal_th, x, y, th) for v, w in self.tentacles]
        best_idx = np.argmin(costs)
        return self.tentacles[best_idx]

    def _smooth_pursuit(self, goal_x, goal_y, goal_th, x, y, th):
        if self.is_goal_reached(goal_x, goal_y, goal_th, x, y, th):
#             print("Goal is within tolerance, stopping planner function")
            return 0.0, 0.0
    
        dx, dy = goal_x - x, goal_y - y
        distance = np.hypot(dx, dy)
        
        # Calculate the angle to the goal
        angle_to_goal = self._angle_between_points((x, y), (goal_x, goal_y))
        
        # Calculate the shortest angular distance -pi to pi
        angle_diff = self._shortest_angular_distance(th, angle_to_goal)
    
        # Calculate linear and angular velocities
        v = self.trapezoidal_profile(distance, self.current_linear_velocity, self.max_linear_velocity, self.max_acceleration)
        w = self.trapezoidal_profile(angle_diff, self.current_angular_velocity, self.max_angular_velocity, self.max_angular_acceleration)
        
        # Adjust linear velocity based on how well we're oriented towards the goal
        orientation_factor = np.cos(angle_diff)
        v *= max(0, orientation_factor)  # Slow down if we're not facing the goal
    
        # If we're close to the goal, start orienting to the final desired orientation
        if distance < self.max_linear_tolerance:
            final_angle_diff = self._shortest_angular_distance(th, goal_th)
            w = self.trapezoidal_profile(final_angle_diff, self.current_angular_velocity, self.max_angular_velocity, self.max_angular_acceleration)
            v = self.trapezoidal_profile(distance, self.current_linear_velocity, self.max_linear_velocity, self.max_acceleration)
        return v, w

    def _rotate_then_drive(self, goal_x, goal_y, goal_th, x, y, th):
        if self.is_goal_reached(goal_x, goal_y, goal_th, x, y, th):
#             print("Goal is within tolerance, stopping planner function")
            return 0.0, 0.0
        
        dx, dy = goal_x - x, goal_y - y
        distance = np.hypot(dx, dy)
        
        # Rotate to face the goal
        if distance < self.max_linear_tolerance:
            # If we're close enough, just rotate to match goal orientation
            target_angle = goal_th
        else:
            target_angle = self._angle_between_points((x, y), (goal_x, goal_y))
        
        # Calculate the shortest angular distance
        angle_diff = self._shortest_angular_distance(th, target_angle)
            
        if abs(angle_diff) > self.max_angular_tolerance:
            w = self.trapezoidal_profile(angle_diff, self.current_angular_velocity, self.max_angular_velocity, self.max_angular_acceleration)
            return 0.0, w
        
        # Now, drive to the goal
        v = self.trapezoidal_profile(distance, self.current_linear_velocity, self.max_linear_velocity, self.max_acceleration)
        w = self.trapezoidal_profile(angle_diff, self.current_angular_velocity, self.max_angular_velocity, self.max_angular_acceleration)
        
        return v, w
    

    def get_control_inputs(self, goal_x, goal_y, goal_th, x, y, th, strategy='rotate_then_drive'):
        if strategy == 'rotate_then_drive':
            v, w = self._rotate_then_drive(goal_x, goal_y, goal_th, x, y, th)
        elif strategy == 'tentacles':
            v, w = self._plan_tentacles(goal_x, goal_y, goal_th, x, y, th)
        elif strategy == "smooth_pursuit":
            v, w = self._smooth_pursuit(goal_x, goal_y, goal_th, x, y, th)
        else:
            raise ValueError("Invalid strategy. Must be 'rotate_then_drive' or 'tentacles' or 'smooth_pursuit'.")
        
        self.current_linear_velocity = v
        self.current_angular_velocity = w
        return {
            'linear_velocity': v,
            'angular_velocity': w,
            'current_x': x,
            'current_y': y,
            'current_theta': th,
            'goal_x': goal_x,
            'goal_y': goal_y,
            'goal_theta': goal_th
        }

    def visualize_rotate_then_drive(self, goal_x, goal_y, goal_th, x, y, th, steps=100):
        plt.figure(figsize=(10, 10))
        plt.scatter(x, y, color='green', s=100, label='Start')
        plt.scatter(goal_x, goal_y, color='blue', s=100, label='Goal')

        x_traj, y_traj = [x], [y]
        th_current = th

        for _ in range(steps):
            v, w = self._rotate_then_drive(goal_x, goal_y, goal_th, x_traj[-1], y_traj[-1], th_current)
            th_current += w * self.dt
            x_new = x_traj[-1] + v * np.cos(th_current) * self.dt
            y_new = y_traj[-1] + v * np.sin(th_current) * self.dt
            x_traj.append(x_new)
            y_traj.append(y_new)

            if self.is_goal_reached(goal_x, goal_y, goal_th, x_new, y_new, th_current):
                break

        plt.plot(x_traj, y_traj, 'g-', linewidth=2, label='Path')
        plt.legend()
        plt.title('Rotate-then-Drive Path')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.grid(True)
        plt.axis('equal')
        plt.show()

    def visualize_tentacles(self, x, y, th, extrapolation_steps=100):
        plt.figure(figsize=(12, 12))
        plt.scatter(x, y, color='green', s=100, label='Start')

        colors = plt.cm.rainbow(np.linspace(0, 1, len(self.tentacles)))

        for (v, w), color in zip(self.tentacles, colors):
            x_traj, y_traj = [x], [y]
            th_traj = th
            for _ in range(self.steps):
                x_new = x_traj[-1] + self.dt * v * np.cos(th_traj) * extrapolation_steps
                y_new = y_traj[-1] + self.dt * v * np.sin(th_traj) * extrapolation_steps
                x_traj.append(x_new)
                y_traj.append(y_new)
                th_traj += w * self.dt

                if _ == self.steps // 2:
                    dx, dy = x_new - x_traj[-2], y_new - y_traj[-2]
                    arrow = Arrow(x_traj[-2], y_traj[-2], dx, dy, width=0.2, color=color)
                    plt.gca().add_patch(arrow)

            plt.plot(x_traj, y_traj, '-', color=color, linewidth=2, label=f'v={v:.2f}, w={w:.2f}')

        if len(self.obstacles) > 0:
            plt.scatter(self.obstacles[:, 0], self.obstacles[:, 1], color='red', s=50, label='Obstacles')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.title('Tentacle Visualization')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.grid(True)
        plt.axis('equal')
        plt.tight_layout()
        plt.show()

    def visualize_plan(self, goal_x, goal_y, goal_th, x, y, th, use_straight_line=False, extrapolation_steps=100):
        v, w = self.plan(goal_x, goal_y, goal_th, x, y, th, use_straight_line)

        plt.figure(figsize=(10, 10))
        plt.scatter(x, y, color='green', s=100, label='Start')
        plt.scatter(goal_x, goal_y, color='blue', s=100, label='Goal')

        x_traj, y_traj = [x], [y]
        th_traj = th
        for _ in range(self.steps):
            x_traj.append(x_traj[-1] + self.dt * v * np.cos(th_traj) * extrapolation_steps)
            y_traj.append(y_traj[-1] + self.dt * v * np.sin(th_traj) * extrapolation_steps)
            th_traj += w * self.dt
        plt.plot(x_traj, y_traj, 'g-', linewidth=2, label='Chosen Path')

        if len(self.obstacles) > 0:
            plt.scatter(self.obstacles[:, 0], self.obstacles[:, 1], color='red', s=50, label='Obstacles')
        plt.legend()
        plt.title(f'Planned Path (v={v:.2f}, w={w:.2f})')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.grid(True)
        plt.axis('equal')
        plt.show()



# # Create an instance of the planner
# obstacles = np.array([[1, 1], [2, 2], [3, 1]])  # Example obstacles
# planner = TentaclePlanner(obstacles, max_linear_velocity=1.0, max_angular_velocity=0.5)

# # Get control inputs for tentacle-based planning
# inputs = planner.get_control_inputs(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0)
# print("Tentacle-based planning:", inputs)

# # Get control inputs for straight-line planning
# inputs_straight = planner.get_control_inputs(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0, use_straight_line=True)
# print("Straight-line planning:", inputs_straight)

# # Visualize the plan
# planner.visualize_plan(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0)
# planner.visualize_plan(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0, use_straight_line=True)

In [ ]:

# # Create an instance of the planner
# obstacles = np.array([[1, 1], [2, 2], [3, 1]])  # Example obstacles
# planner = TentaclePlanner(obstacles, max_linear_velocity=1.0, max_angular_velocity=0.5)

# # Get control inputs for tentacle-based planning
# inputs = planner.get_control_inputs(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0)
# print("Tentacle-based planning:", inputs)

# # Get control inputs for straight-line planning
# inputs_straight = planner.get_control_inputs(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0, use_straight_line=True)
# print("Straight-line planning:", inputs_straight)

# # Visualize the plan
# planner.visualize_plan(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0)
# planner.visualize_plan(goal_x=5, goal_y=5, goal_th=0, x=0, y=0, th=0, use_straight_line=True)